# **Выводы** в конце ноутбука

# REINFORCE

Пусть $X$ это какая-то случайная величина с известным распределением $p_\theta(X)$, где $\theta$ — это параметры модели. Определим функцию награды $J(\theta)$ как

$$ J(\theta) = E[f(X)] = \int_x f(x) p(x) \; dx $$

для произвольной функции $f$, не зависящей от $\theta$.

*Policy gradient* (дословно, градиент стратегии) — это, интуитивно, то направление, куда нужно двигать параметры модели, чтобы функция награды увеличивалась. Алгоритм REINFORCE (почему его всегда пишут капсом автор не знает) на каждом шаге просто определяет policy gradient $\nabla_\theta J(\theta)$ и изменяет параметры в его сторону с каким-то learning rate-ом.

**Зачем это надо**, если есть обычный градиентный спуск? Через policy gradient и reinforcement learning вообще можно оптимизировать более общий класс функций — хоть дискретные (например, можно BLEU для перевода напрямую максимизировать) и даже невычислимые (какие-нибудь субъективные оценки асессоров).

В частности, в таком ключе можно описать игры (для простоты, однопользовательские): есть какие-то награды за совершение каких-то действий (для шахмат: +1 за победу, 0 за ничью, -1 за поражение; для тетриса: +0.1 за «выживание» ещё одну секунду, 1 за удаление слоя, 0 за проигрыш) и нам нужно подобрать такие параметры модели, чтобы максимизировать ожидаемую сумму наград за действия, которые мы совершили, то есть в точности $J(\theta)$.

Теперь немного математики: как нам найти этот $\nabla_\theta J(\theta)$? Оказывается, мы можем выразить его ожидание, а тогда приблизительный градиент можно будет находить сэмплированием и усреднением градиентов — так же, как мы обычно обучаем нейросети на батчах.

$$ \nabla J(\theta) = \nabla E[f(X)] = \int_x f(x) \nabla p(x) \; dx = \int_x f(x) p(x) \nabla_\theta \log p(x) \; dx = E[f(x) \nabla_\theta \log p(x)] $$

Переход между 2 и 3 верен, потому что $\nabla \log p(x) = \frac{\nabla p(x)}{p(x)}$ (просто подставьте и $p(x)$ сократится). Это называют log-derivative trick.

Мы научились получить приблизительный градиент через сэмплирование. Давайте теперь что-нибудь обучим.

## `CartPole-v0`

Про задачу можно почитать тут: https://gym.openai.com/envs/CartPole-v0/. Tl;dr: есть вертикально стоящая палка на подвижной платформе; нужно двигать платформу так, чтобы палка не упала.

<img width='500px' src='https://preview.redd.it/sqjzj2cgnpt21.gif?overlay-align=bottom,left&overlay-pad=8,16&crop=1200:628.272251309,smart&overlay-height=0.10&overlay=%2Fv9vyirk6hl221.png%3Fs%3Db466421949eb723078743745ce6421609d7a9c66&width=1200&height=628.272251309&s=ba84ac5a9c14946456808c15f2754cb7369b8de9'>

OpenAI в 2016-м году выпустили `gym` — библиотечку для абстрагирования RL-ных сред от алгоритма. Есть абстрактная *среда* (`env`), в ней есть какие-то *состояния* (`state`), из каждого состояния есть какой-то фиксированный набо *действий* (`action`), ведущих (возможно, с какими-то вероятностями) в другие состояния, и за разные действия в разных состояниях дается какая-то *награда* (`reward`). Как конкретно устроена игра, нам думать не нужно.

In [91]:
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [92]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

#plt.imshow(env.render("rgb_array"))
env.close()

In [93]:
state_dim # 4 состояния системы:

(4,)

In [94]:
n_actions

2

## Сеть

Для REINFORCE вам нужна модель, которая берёт на вход состояние (каким-то образом закодированное) и возвращает вероятностное распределение действий в нём.

Старайтесь не перемудрить — в общем случае сети для RL могут быть [довольно сложными](https://d4mucfpksywv.cloudfront.net/research-covers/openai-five/network-architecture.pdf), но CartPole не стоит того, чтобы писать глубокие архитектуры.

In [95]:
import torch
import torch.nn as nn

In [96]:
agent = nn.Sequential(
    nn.Linear(4, 32),
    nn.Sigmoid(),
    nn.Linear(32, 2),
    nn.LogSoftmax(dim=1)  # важно, что на выходе должны быть не вероятности, а логиты
)

OpenAI Gym работает с numpy, а не напрямую с фреймворками. Для удобства, напишите функцию-обёртку, которая принимает состояния (`numpy array` размера `[batch, state_shape]`) и возвращает вероятности (размера `[batch, n_actions]]`, должны суммироваться в единицу).

In [97]:
def predict_proba(states):
    states = torch.Tensor(states)
    logits = agent(states)
    # сконвертируйте состояния в тензор
    # вычислите логиты
    # вызовите софтмакс, чтобы получить веряотности
    return logits.exp().detach().numpy() # не bottleneck, поэтому так можно написать, но лучше torch.exp(logits...)

In [98]:
test_states = np.array([env.reset() for _ in range(5)])
test_probas = predict_proba(test_states)
assert isinstance(test_probas, np.ndarray), "you must return np array and not %s" % type(test_probas)
assert tuple(test_probas.shape) == (test_states.shape[0], n_actions), "wrong output shape: %s" % np.shape(test_probas)
assert np.allclose(np.sum(test_probas, axis = 1), 1), "probabilities do not sum to 1"

## Тестовый прогон

Хоть наша модель не обучена, её уже можно использовать, чтобы играть в произвольной среде.

In [99]:
def generate_session(t_max=1000):
    """ 
    Играет одну сессию REINFORCE-агентом и возвращает последовательность состояний,
    действий и наград, которые потом будут использоваться при обучении.
    """
    
    # тут будем хранить сессию
    states, actions, rewards = [],[],[]
    
    s = env.reset()
    
    for t in range(t_max):
        
        # вероятности следующих действий, aka p(a|s)
        action_probas = predict_proba(np.array([s]))[0] 
        
        # сэмплируйте оттуда действие (посказка: np.random.choice)
        a = np.random.choice([0, 1], p=action_probas)
        
        new_s, r, done, info = env.step(a) # info здесь не используется
        
        states.append(s)
        actions.append(a)
        rewards.append(r)
        
        s = new_s
        if done:
            break
            
    return states, actions, rewards

In [100]:
# протестируйте
states, actions, rewards = generate_session()

In [101]:
states

[array([ 0.0181061 , -0.04347813,  0.02581502, -0.00647325]),
 array([ 0.01723654,  0.15126427,  0.02568556, -0.29090075]),
 array([ 0.02026182, -0.04421434,  0.01986754,  0.00977116]),
 array([ 0.01937753, -0.2396155 ,  0.02006296,  0.30865573]),
 array([ 0.01458522, -0.04478508,  0.02623608,  0.02236703]),
 array([ 0.01368952,  0.14995099,  0.02668342, -0.26192403]),
 array([ 0.01668854, -0.04554149,  0.02144494,  0.0390543 ]),
 array([ 0.01577771,  0.14926648,  0.02222603, -0.24678614]),
 array([ 0.01876304, -0.04616573,  0.0172903 ,  0.0528237 ]),
 array([ 0.01783973,  0.14870408,  0.01834678, -0.23435424]),
 array([ 0.02081381, -0.04667513,  0.01365969,  0.06405891]),
 array([ 0.01988031,  0.14824833,  0.01494087, -0.22428319]),
 array([ 0.02284527,  0.34315359,  0.01045521, -0.51221599]),
 array([ 2.97083454e-02,  5.38126730e-01,  2.10886368e-04, -8.01585939e-01]),
 array([ 0.04047088,  0.34300189, -0.01582083, -0.50883668]),
 array([ 0.04733092,  0.53834312, -0.02599757, -0.8064

In [102]:
actions

[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]

In [103]:
rewards

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

### Computing cumulative rewards

In [104]:
def get_cumulative_rewards(rewards, gamma=0.99):
    """
    Принимает массив ревардов и возвращает discounted массив по следующей формуле:
    
        G_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    Тут нет ничего сложного -- итерируйтесь от последнего до первого
    и насчитывайте G_t = r_t + gamma*G_{t+1} рекуррентно.
    """
    
    g = 0
    g_t = []
    for r in reversed(rewards):
      g = r + gamma * g
      g_t.append(g)
    return np.array(list(reversed(g_t)))

In [105]:
get_cumulative_rewards(rewards)
assert len(get_cumulative_rewards(list(range(100)))) == 100
assert np.allclose(get_cumulative_rewards([0, 0, 1, 0, 0, 1, 0], gamma=0.9),
                   [1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, -2, 3, -4, 0], gamma=0.5),
                   [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0, 0, 1, 2, 3, 4, 0], gamma=0),
                   [0, 0, 1, 2, 3, 4, 0])
print("Вроде норм")

Вроде норм


#### Loss function and updates

Вспомним, что нам нужно оптимизировать

$$ J \approx  { 1 \over N } \sum  _{s_i,a_i} \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$


Используя REINFORCE, нам в алгоритме по сути нужно максимизировать немного другую функцию:

$$ \hat J \approx { 1 \over N } \sum  _{s_i,a_i} log \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$

Когда мы будем вычислять её градиент, мы получим в точности policy gradient из REINFORCE.

In [106]:
def to_one_hot(y_tensor, n_dims=None):
    """ Конвертирует целочисленный вектор в one-hot матрицу. """
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [107]:
optimizer = torch.optim.Adam(agent.parameters())
# тут определите оптимизатор для модели
# например, Adam с дефолтными параметрами

# натрениться с конкретной функции
def train_on_session(states, actions, rewards, gamma = 0.99):
    optimizer.zero_grad()

    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int32)
    cumulative_returns = np.array(get_cumulative_rewards(rewards, gamma))
    cumulative_returns = torch.tensor(cumulative_returns, dtype=torch.float32)
    
    logprobas = agent(states)
    probas = logprobas.exp()
    
    assert all(isinstance(v, torch.Tensor) for v in [probas, logprobas]), \
        "please use compute using torch tensors and don't use predict_proba function"
    
    # выберем и просуммируем лог-вероятности только для выбранных действий
    # аналог кросс-энтропии
    logprobas_for_actions = torch.sum(logprobas * to_one_hot(actions), dim=1)
    
    J_hat = torch.mean(cumulative_returns * logprobas_for_actions) # берем среднее от итераций, которые совершали # формула для REINFORCE
    

    # опционально: энтропийная регуляризация
    # не регуляризация, а способ не скатываться в локальный минимум
    entropy_reg = torch.mean(logprobas * probas) # вычислите среднюю энтропию вероятностей; не забудьте знак!
    
    loss = - J_hat - 0.1 * entropy_reg
    
    loss.backward()
    optimizer.step()
    # шагните в сторону градиента
    # ....
    
    # верните ревард сессии, чтобы потом их печатать
    return np.sum(rewards)

## Само обучение

In [108]:
# играют на батче независимых сессий

for i in range(100):
    
    rewards = [train_on_session(*generate_session()) for _ in range(100)]
    
    print (i, " mean reward:%.3f"%(np.mean(rewards)))

    if np.mean(rewards) > 500:
        print ("Победа!")
        break

# выводит количество тиков, которое мы продержались

0  mean reward:21.440
1  mean reward:19.620
2  mean reward:23.910
3  mean reward:20.920
4  mean reward:18.170
5  mean reward:24.000
6  mean reward:23.970
7  mean reward:23.350
8  mean reward:25.150
9  mean reward:22.790
10  mean reward:27.130
11  mean reward:29.330
12  mean reward:29.760
13  mean reward:33.150
14  mean reward:32.420
15  mean reward:32.160
16  mean reward:33.410
17  mean reward:39.270
18  mean reward:36.510
19  mean reward:43.410
20  mean reward:40.690
21  mean reward:47.880
22  mean reward:47.470
23  mean reward:54.420
24  mean reward:46.970
25  mean reward:52.620
26  mean reward:68.330
27  mean reward:64.650
28  mean reward:60.230
29  mean reward:62.810
30  mean reward:82.920
31  mean reward:100.550
32  mean reward:88.540
33  mean reward:115.690
34  mean reward:149.640
35  mean reward:137.450
36  mean reward:225.920
37  mean reward:173.110
38  mean reward:207.510
39  mean reward:138.170
40  mean reward:163.730
41  mean reward:211.500
42  mean reward:276.020
43  mean r

## Видосик

In [ ]:
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),directory="videos",force=True)
# sessions = [generate_session() for _ in range(100)]
sessions = []
for _ in range(100):
  states, actions, rewards = generate_session()
  sessions.append((states, actions, rewards))
env.close()

In [ ]:
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1]))

# **Выводы:**

1. Мы победили! 503.5 балла